In [1]:
%matplotlib inline               
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('la data without aqi.csv')

In [3]:
df.head()

,Avg PM2.5,Avg Max Temp,Avg Dew Temp,Avg Wind Speed,Avg Humidity
0,8.642857,64.857143,33.571429,9.142857,48.357143
1,36.614286,64.000000,40.000000,6.571429,60.500000
2,24.628571,77.428571,36.714286,6.428571,44.142857
3,9.300000,84.000000,27.428571,7.142857,25.571429
4,11.485714,68.142857,42.285714,7.571429,54.571429


In [4]:
df.shape

(449, 5)

In [5]:
X = df.drop('Avg PM2.5',axis=1)
y = df['Avg PM2.5']

In [6]:
df = df.reset_index()
df = df.dropna()

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
X_train.shape

(359, 4)

In [9]:
X_test.shape

(90, 4)

In [10]:
y_train.shape

(359,)

In [11]:
y_test.shape

(90,)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

In [16]:
n=3 # how many times to shuffle the training data
nhn_range=[1,2,4,6,8,10] # number of hidden neurons

In [17]:
score_dict = {}
for nhn in nhn_range:
    mlp = MLPRegressor(hidden_layer_sizes=(nhn,), activation='relu', 
                       solver='adam', shuffle=False, random_state=42, 
                       max_iter=10000, momentum=0.7, early_stopping=True, 
                       validation_fraction=0.15)


    nhn_scores = []
    for _ in range(n):

        X_train = shuffle(X_train)
        score = np.sqrt(-cross_val_score(mlp, X_train, 
                    y_train, 
                    cv=5, scoring='neg_mean_squared_error')).mean()
        nhn_scores.append(score)
    score_dict[nhn] = nhn_scores

In [18]:
score_df = pd.DataFrame.from_dict(score_dict)
print(score_df)

         1         2         4         6         8         10
0  5.653349  5.951574  5.879272  6.454201  6.319244  6.803037
1  5.655978  5.797924  6.082612  6.283480  6.352889  6.804382
2  5.655375  5.933327  6.134863  6.477393  6.075155  7.046039


In [19]:
score_dict = {}
for nhn in nhn_range:
    mlp = MLPRegressor(hidden_layer_sizes=(nhn,), activation='relu', 
                       solver='adam', shuffle=False, random_state=42, 
                       max_iter=2000, momentum=0.7, early_stopping=True, 
                       validation_fraction=0.15)


    nhn_scores = []
    for _ in range(n):

        X_train = shuffle(X_train)
        mlp.fit(X_train,y_train)
        score = mlp.score(X_test, y_test).mean()
        nhn_scores.append(score)
    score_dict[nhn] = nhn_scores

C:\Users\theda\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\theda\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [20]:
score_df = pd.DataFrame.from_dict(score_dict)

In [21]:
print(score_df)

        10        50        100       200       400       600
0 -0.126465 -0.340855 -0.203829 -0.068093 -0.101070  0.004482
1  0.027626 -0.480756 -0.308382 -0.144247 -0.088470 -0.137548
2 -0.075815 -0.307181 -0.168696  0.008988 -0.289974 -0.103535


In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
score_dict = {}
for nhn in nhn_range:
    mlp = MLPRegressor(hidden_layer_sizes=(nhn,), activation='relu', 
                       solver='adam', shuffle=False, random_state=42, 
                       max_iter=2000, momentum=0.7, early_stopping=True, 
                       validation_fraction=0.15)


    nhn_scores = []
    for _ in range(n):

        X_train = shuffle(X_train)
        mlp.fit(X_train,y_train)
        predictions = mlp.predict(X_test)
        score = mean_squared_error(y_test, predictions).mean()
        nhn_scores.append(score)
    score_dict[nhn] = nhn_scores

In [24]:
score_df = pd.DataFrame.from_dict(score_dict)
print(score_df)

         10         50         100        200        400        600
0  32.635013  38.794006  31.298572  31.893421  34.636826  30.561701
1  38.225211  39.457273  35.254558  31.644332  37.894691  29.786396
2  30.825742  36.335435  34.837430  32.150797  37.743133  32.063022
